<a href="https://colab.research.google.com/github/chagas98/ProgrammingStudies/blob/main/Pytorch_Chap12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning with pytorch and scikit-learn

_Chapter 12_ - Parallelizing Neural Network Training with PyTorch

### **Tensors**

Tensors can be define as a rank-0 tensor for scalars, a vector as a rank-1 tensor, a matrix as a rank-2 tensor, and matrices stacked in a third dimension can be defined as rank-3 tensors.

Tensors in Pytorch are simular to NumPy arrays, except that tensors are optimized for **automatic differentiation** and can run on GPUs


In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-c

#### Code with Tensors

We can simply create a tensor from a list or a NumPy array using torch.tensor or the torch.from_numpy

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
# Creating
a = [1, 2, 3]
b = np.array([4, 5, 6], dtype=np.int32)
t_a = torch.tensor(a)
t_b = torch.from_numpy(b)
print(t_a)
print(t_a.shape)
print(t_b)
print(t_b.shape)

# Creating a tensor of
#random values
rand_tensor = torch.randn(2,3)

# standard normal distribution
rand_tensor = torch.randn(2,3)

# integers
rand_tensor = torch.randint(low=10, high=20, size=(2,3))

print(rand_tensor)

tensor([1, 2, 3])
torch.Size([3])
tensor([4, 5, 6], dtype=torch.int32)
torch.Size([3])
tensor([[14, 18, 17],
        [15, 14, 14]])


In [ ]:
# Named Tensors
batch_t = torch.randn(2, 3, 5, 5) # (batch, channels, rows, columns)
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])

img_gray_naive = img_t.mean(-3) # calculate mean from channels
batch_gray_naive = batch_t.mean(-3) # calculate mean from channels
img_gray_naive.shape, batch_gray_naive.shape


(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [ ]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

##### Data type and shape

Numbers in python are objects, converting the number into a fill-fledged python object, a problem called _boxing_ when allocating millions of numbers. Lists have the same problem. Compiled language has more advantages (NumPy and PyTorch are alternatives)

- dtype argument to tensor constructors specifies the numerical data type that bill be contnained in the tensor;

- neural networks are typically executed with 32-bit floating-point precision;

- 64-bit will not buy improvements in the accuracy of a model and will require more memory and cocmputing time.

- assined dtype with torch.double, torch.int64, etc.


##### Operations



In [ ]:
# change the data type
print('dtype')
t_a_new = t_a.to(torch.int64)
print(t_a_new.dtype)

# Transposing
print('\ntransposing')
t = torch.rand(3, 5)
t_tr = torch.transpose(t, 0, 1)
t_tr2 = t.transpose(0, 1)
print(t.shape, ' --> ', t_tr.shape, 'or', t_tr2.shape)

print('\nreshape')
t = torch.zeros(30)
print(t.shape)
t_reshape = t.reshape(5, 6)
print(t_reshape.shape)

## Squeeze - Removing the unnecessary dimensions
print('\nsqueeze')
t = torch.zeros(1, 2, 1, 4, 1)
t_sqz = torch.squeeze(t, 2)
print(t.shape, ' --> ', t_sqz.shape)

# Matrix - Matrix Product
print('\ndot product')
torch.manual_seed(1)
t1 = 2 * torch.rand(5, 2) - 1
t2 = torch.normal(mean=0, std=1, size=(5, 2))
t3 = torch.matmul(t1, torch.transpose(t2, 0, 1))
print(t3)

# Split in different sizes
print('\nSplit')
torch.manual_seed(1)
t = torch.rand(5)
t_splits = torch.split(t, split_size_or_sections=[3, 2])
[item.numpy().shape for item in t_splits]

#Cat and stack remains the same as numpy

dtype
torch.int64

transposing
torch.Size([3, 5])  -->  torch.Size([5, 3]) or torch.Size([5, 3])

reshape
torch.Size([30])
torch.Size([5, 6])

squeeze
torch.Size([1, 2, 1, 4, 1])  -->  torch.Size([1, 2, 4, 1])

dot product
tensor([[ 0.1312,  0.3860, -0.6267, -1.0096, -0.2943],
        [ 0.1647, -0.5310,  0.2434,  0.8035,  0.1980],
        [-0.3855, -0.4422,  1.1399,  1.5558,  0.4781],
        [ 0.1822, -0.5771,  0.2585,  0.8676,  0.2132],
        [ 0.0330,  0.1084, -0.1692, -0.2771, -0.0804]])

Split


[(3,), (2,)]

### **DataLoaders**

When dataset is too large to fit into the compute memory, we will need to load the data from the main storage device in **chunks** (**batch by batch**)

_torch.utils.data.DataLoader_ create a DataLoader class, which we can use to iterate through the individual elements in the input dataset.

In [ ]:
from torch.utils.data import DataLoader

t = torch.arange(6, dtype=torch.float32)
data_loader = DataLoader(t, batch_size=3, drop_last=False) #drop_last remove last batch when not divisible

for i, batch in enumerate(data_loader, 1):
  print(f'batch {i}:', batch)

batch 1: tensor([0., 1., 2., 3.])


#### Combining two tensors into a joint dataset

A custm Dataset class must contain the following methods:

- \_init\_(): this is where the initial logic happens, suh as reading existing arrays, loading a file, filtering data, and so on.

- \_getitem\_(): this returns the corresponding sample to the given index

In [ ]:
from torch.utils.data import Dataset

class JointDataset(Dataset):
  def __init__(self, x, y):
    self.x = x
    self.y = y

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

torch.manual_seed(1)
t_x = torch.rand([4, 3], dtype=torch.float32)
t_y = torch.arange(4)

joint_dataset = JointDataset(t_x, t_y)

for example in joint_dataset:
  #print(' x: ', example[0], ' y: ', example[1])
  print(example)

(tensor([0.7576, 0.2793, 0.4031]), tensor(0))
(tensor([0.7347, 0.0293, 0.7999]), tensor(1))
(tensor([0.3971, 0.7544, 0.5695]), tensor(2))
(tensor([0.4388, 0.6387, 0.5247]), tensor(3))
